In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [2]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex"
today

'day15'

In [3]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines

def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid

def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]

def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]

def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [4]:
with open(f"./data/{today}.txt") as f:
    lines = f.read().rstrip()

grid_init = [[c for c in line.rstrip()] for line in lines.split("\n\n")[0].split("\n")]
instructions = [line.rstrip() for line in lines.split("\n\n")[1].split("\n")]

robot_start = None
for i, row in enumerate(grid_init):
    if "@" in row:
        robot_start = (i, row.index("@"))
        break
assert robot_start is not None

len(grid_init), len(grid_init[0]), robot_start, len(instructions), len(instructions[0])

(50, 50, (24, 24), 20, 1000)

In [5]:
def move_robot(direction: str, pos: tuple[int, int], grid: list[list[str]]) -> tuple[int, int]:
    d = {"^": 0, ">": 1, "v": 2, "<": 3}[direction]
    i = 1
    while grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] == "O":
        i += 1
    if grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] == "#":
        return pos
    assert grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] == "."
    while i > 1:
        grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] = "O"
        i -= 1
    grid[pos[0] + dirs4[d][0]][pos[1] + dirs4[d][1]] = "@"
    grid[pos[0]][pos[1]] = "."
    return (pos[0] + dirs4[d][0], pos[1] + dirs4[d][1])

In [6]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))

In [7]:
grid = deepcopy(grid_init)
pos = robot_start

for moves in instructions:
    for move in moves:
        pos = move_robot(move, pos, grid)  # type: ignore

In [8]:
s = 0
for r, row in enumerate(grid):
    for c, char in enumerate(row):
        if char == "O":
            s += 100*r + c
s

1383666

In [9]:
expanded_grid: list[list[str]] = []
for row in grid_init:
    expanded_row = []
    for c in row:
        if c == "#":
            expanded_row.extend(["#", "#"])
        elif c == "O":
            expanded_row.extend(["[", "]"])
        elif c == ".":
            expanded_row.extend([".", "."])
        elif c == "@":
            expanded_row.extend(["@", "."])
        else:
            raise Exception("huh?")
    expanded_grid.append(expanded_row.copy())

robot_start_2 = None
for i, row in enumerate(expanded_grid):
    if "@" in row:
        robot_start_2 = (i, row.index("@"))
        break
assert robot_start_2 is not None

robot_start_2

(24, 48)

In [10]:
def push_boxes(sign: int, pos: tuple[int, int], grid: list[list[str]], dry_run: bool = True) -> bool:
    if grid[pos[0] + sign][pos[1]] == "#" or grid[pos[0] + sign][pos[1] + 1] == "#":
        return False
    if grid[pos[0] + sign][pos[1]] == "]" and not push_boxes(sign, (pos[0] + sign, pos[1] - 1), grid, dry_run):
        return False
    if grid[pos[0] + sign][pos[1]] == "[" and not push_boxes(sign, (pos[0] + sign, pos[1]), grid, dry_run):
        return False
    if grid[pos[0] + sign][pos[1] + 1] == "[" and not push_boxes(sign, (pos[0] + sign, pos[1] + 1), grid, dry_run):
        return False
    
    if not dry_run:
        grid[pos[0] + sign][pos[1]] = "["
        grid[pos[0] + sign][pos[1] + 1] = "]"
        grid[pos[0]][pos[1]] = "."
        grid[pos[0]][pos[1] + 1] = "."

    return True


def move_robot_2(direction: str, pos: tuple[int, int], grid: list[list[str]]) -> tuple[int, int]:
    if direction in ["<", ">"]:
        d = {"^": 0, ">": 1, "v": 2, "<": 3}[direction]
        i = 1
        while grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] in ["[", "]"]:
            i += 1
        if grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] == "#":
            return pos
        assert grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] == "."
        while i > 1:
            grid[pos[0] + i*dirs4[d][0]][pos[1] + i*dirs4[d][1]] = grid[pos[0] + (i-1)*dirs4[d][0]][pos[1] + (i-1)*dirs4[d][1]]
            i -= 1
        grid[pos[0] + dirs4[d][0]][pos[1] + dirs4[d][1]] = "@"
        grid[pos[0]][pos[1]] = "."
        return (pos[0] + dirs4[d][0], pos[1] + dirs4[d][1])
    
    if direction == "^":
        sign = -1
    else:
        sign = 1
    if grid[pos[0] + sign][pos[1]] == "#":
        return pos
    if grid[pos[0] + sign][pos[1]] == ".":
        grid[pos[0] + sign][pos[1]] = "@"
        grid[pos[0]][pos[1]] = "."
        return (pos[0] + sign, pos[1])
    if grid[pos[0] + sign][pos[1]] == "[":
        if not push_boxes(sign, (pos[0] + sign, pos[1]), grid):
            return pos
        push_boxes(sign, (pos[0] + sign, pos[1]), grid, False)
        grid[pos[0] + sign][pos[1]] = "@"
        grid[pos[0]][pos[1]] = "."
        return (pos[0] + sign, pos[1])
    if grid[pos[0] + sign][pos[1]] == "]":
        if not push_boxes(sign, (pos[0] + sign, pos[1] - 1), grid):
            return pos
        push_boxes(sign, (pos[0] + sign, pos[1] - 1), grid, False)
        grid[pos[0] + sign][pos[1]] = "@"
        grid[pos[0]][pos[1]] = "."
        return (pos[0] + sign, pos[1])

    raise Exception("should never end up here")

In [11]:
grid = deepcopy(expanded_grid)
pos = robot_start_2

for moves in instructions:
    for move in moves:
        pos = move_robot_2(move, pos, grid)  # type: ignore

In [12]:
s = 0
for r, row in enumerate(grid):
    for c, char in enumerate(row):
        if char == "[":
            s += 100*r + c
s

1412866